<a href="https://colab.research.google.com/github/mayankcharde/Fire-Smoke-Detection-repo/blob/main/SmokeFireDetection(Vedio).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ===========================================================
# FAST FIRE & SMOKE TRAINING + VIDEO PREDICTION (YOLOv8n)
# - Uses /content/bucket11.mp4, /content/printer31.mp4
# - Skips roomfire41.mp4 for training
# - Uses /content/markup.json
# - Downsamples frames, smaller imgsz, fewer epochs
# ===========================================================

!pip install ultralytics opencv-python tqdm --quiet

import os
import json
import random
import cv2
from tqdm import tqdm
from ultralytics import YOLO
import numpy as np
from shutil import copy2
import shutil
from google.colab import files
import torch

print("PyTorch CUDA available:", torch.cuda.is_available())

# ------------------ FAST TRAINING HYPERPARAMS ------------------
IMG_SIZE = 512          # smaller than 640 -> faster
EPOCHS = 10             # fewer epochs -> faster
BATCH_SIZE = 8
MAX_FRAMES_PER_VIDEO = 300   # limit annotated frames per video
EXCLUDED_VIDEOS = {"roomfire41"}   # not used for training
CLASS_MAP = {"fire": 0, "smoke": 1}

# ===========================================================
# 1️⃣ Paths
# ===========================================================
DATA_DIR   = "/content"
VIDEO_DIR  = "/content"
MARKUP_PATH = "/content/markup.json"

print("Files in /content:", os.listdir("/content"))
if not os.path.exists(MARKUP_PATH):
    raise FileNotFoundError("markup.json not found in /content. Upload it first.")

# Clean previous dataset folder if exists
YOLO_DATA_DIR = "/content/yolo_fire_smoke"
if os.path.exists(YOLO_DATA_DIR):
    shutil.rmtree(YOLO_DATA_DIR)

IMAGES_DIR = os.path.join(YOLO_DATA_DIR, "images")
LABELS_DIR = os.path.join(YOLO_DATA_DIR, "labels")
os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(LABELS_DIR, exist_ok=True)

# ===========================================================
# 2️⃣ Load markup.json
# ===========================================================
with open(MARKUP_PATH, "r") as f:
    markup = json.load(f)

print("Videos in markup:", list(markup.keys()))
print("Excluded from training:", EXCLUDED_VIDEOS)
print("Class mapping:", CLASS_MAP)

def extract_frame(cap, frame_idx):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    return frame if ret else None

# ===========================================================
# 3️⃣ Build YOLO dataset (with FRAME SUBSAMPLING)
# ===========================================================
all_samples = []

for video_key, frames in markup.items():
    if video_key in EXCLUDED_VIDEOS:
        print(f"\n[INFO] Skipping {video_key} (excluded from training)")
        continue

    video_path = os.path.join(VIDEO_DIR, f"{video_key}.mp4")
    if not os.path.exists(video_path):
        print(f"[WARN] Missing video file for key '{video_key}': {video_path}")
        continue

    print(f"\nProcessing {video_path} with {len(frames)} annotated frames...")

    # ---- Subsample frames for speed ----
    if len(frames) > MAX_FRAMES_PER_VIDEO:
        frames = random.sample(frames, MAX_FRAMES_PER_VIDEO)
        print(f"[INFO] Subsampled to {len(frames)} frames for faster training")

    cap = cv2.VideoCapture(video_path)

    for f in tqdm(frames, desc=f"{video_key} frames"):
        w, h = f["width"], f["height"]
        frame_num = f["frame_num"]

        frame = extract_frame(cap, frame_num)
        if frame is None:
            continue

        img_name = f"{video_key}_frame{frame_num}.jpg"
        img_path = os.path.join(IMAGES_DIR, img_name)

        # Optional: resize frame down before saving (YOLO will resize again,
        # but this saves disk & IO, makes things a bit lighter)
        frame_resized = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        cv2.imwrite(img_path, frame_resized)

        label_path = os.path.join(LABELS_DIR, img_name.replace(".jpg", ".txt"))
        lines = []

        for obj in f["objects"]:
            cls = CLASS_MAP.get(obj["class"], None)
            if cls is None:
                continue

            x1, y1, x2, y2 = obj["x1"], obj["y1"], obj["x2"], obj["y2"]

            # NOTE: normalization still uses original w, h from markup
            xc = (x1 + x2) / 2 / w
            yc = (y1 + y2) / 2 / h
            bw = (x2 - x1) / w
            bh = (y2 - y1) / h

            lines.append(f"{cls} {xc} {yc} {bw} {bh}")

        with open(label_path, "w") as lf:
            lf.write("\n".join(lines))

        all_samples.append((img_path, label_path))

    cap.release()

print("\nTotal samples after subsampling:", len(all_samples))
if len(all_samples) == 0:
    raise RuntimeError("No samples created. Check markup.json and filenames.")

# ===========================================================
# 4️⃣ Train/Val split
# ===========================================================
random.shuffle(all_samples)
split_idx = int(len(all_samples) * 0.8)
train_samples = all_samples[:split_idx]
val_samples   = all_samples[split_idx:]

for split_type in ["train", "val"]:
    os.makedirs(os.path.join(IMAGES_DIR, split_type), exist_ok=True)
    os.makedirs(os.path.join(LABELS_DIR, split_type), exist_ok=True)

def move_samples(samples, split_name):
    for img, label in samples:
        copy2(img, os.path.join(IMAGES_DIR, split_name, os.path.basename(img)))
        copy2(label, os.path.join(LABELS_DIR, split_name, os.path.basename(label)))

move_samples(train_samples, "train")
move_samples(val_samples, "val")

print(f"Train samples: {len(train_samples)} | Val samples: {len(val_samples)}")

# ===========================================================
# 5️⃣ Create YOLO data config
# ===========================================================
yaml_path = os.path.join(YOLO_DATA_DIR, "fire_smoke.yaml")
with open(yaml_path, "w") as f:
    f.write(f"""
path: {YOLO_DATA_DIR}
train: images/train
val: images/val
names:
  0: fire
  1: smoke
""")

print("\nYOLO data config:")
print(open(yaml_path).read())

# ===========================================================
# 6️⃣ Train YOLOv8n (fast)
# ===========================================================
model = YOLO("yolov8n.pt")
results = model.train(
    data=yaml_path,
    epochs=EPOCHS,         # faster
    imgsz=IMG_SIZE,        # smaller -> faster
    batch=BATCH_SIZE,
    workers=2,             # small but helps
    name="fire_smoke_fast",
    project=YOLO_DATA_DIR
)

MODEL_PATH = os.path.join(YOLO_DATA_DIR, "fire_smoke_fast", "weights", "best.pt")
print("\nBest model weights at:", MODEL_PATH)

# ===========================================================
# 7️⃣ Detection on any uploaded video (you can use roomfire41.mp4)
# ===========================================================
print("\nUpload a video for detection (e.g., roomfire41.mp4):")
uploaded = files.upload()
test_video = list(uploaded.keys())[0]
input_video = f"/content/{test_video}"
print("Running detection on:", input_video)

cap = cv2.VideoCapture(input_video)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS) or 25.0

out_path = "/content/output_detection_fast.mp4"
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

detect_model = YOLO(MODEL_PATH)
colors = {0: (0, 0, 255), 1: (255, 0, 0)}  # fire=red, smoke=blue
labels = {0: "fire", 1: "smoke"}

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = detect_model(frame, conf=0.4, verbose=False)[0]
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        c = int(box.cls.cpu().numpy())
        conf = float(box.conf.cpu().numpy())
        lbl = f"{labels.get(c, 'obj')} {conf:.2f}"
        color = colors.get(c, (0, 255, 0))

        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(frame, lbl, (int(x1), int(y1) - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out.write(frame)

cap.release()
out.release()

print("\n✅ Fast detection video saved at:", out_path)
print("Download or play it from the Files panel.")


PyTorch CUDA available: False
Files in /content: ['.config', 'bucket11 (1).mp4', 'yolov8n.pt', 'output_detection_fast.mp4', 'bucket11.mp4', 'markup.json', 'printer31.mp4', 'yolo_fire_smoke', 'sample_data']
Videos in markup: ['bench11', 'bench12', 'bench21', 'bench22', 'bench31', 'boiler11', 'boiler12', 'boiler21', 'boiler22', 'boiler31', 'boiler32', 'bucket11', 'bucket12', 'bucket22', 'bucket23', 'bucket31', 'bucket32', 'bucket33', 'bucket42', 'corridor11', 'corridor21', 'corridor22', 'door11', 'door22', 'door33', 'door41', 'door52', 'door63', 'door74', 'fire11', 'fire22', 'fire31', 'fire42', 'gas11', 'gas12', 'gas21', 'gas22', 'gas31', 'gas32', 'office11', 'office12', 'office21', 'office22', 'office31', 'office42', 'officebucket11', 'officebucket21', 'officebucket22', 'officebucket31', 'officebucket32', 'printer11', 'printer12', 'printer21', 'printer22', 'printer31', 'printer32', 'roomfire11', 'roomfire22', 'roomfire31', 'roomfire41', 'roomsmoke11', 'roomsmoke22', 'roomsmoke32', 'room

bucket11 frames: 100%|██████████| 76/76 [00:12<00:00,  5.88it/s]


[WARN] Missing video file for key 'bucket12': /content/bucket12.mp4
[WARN] Missing video file for key 'bucket22': /content/bucket22.mp4
[WARN] Missing video file for key 'bucket23': /content/bucket23.mp4
[WARN] Missing video file for key 'bucket31': /content/bucket31.mp4
[WARN] Missing video file for key 'bucket32': /content/bucket32.mp4
[WARN] Missing video file for key 'bucket33': /content/bucket33.mp4
[WARN] Missing video file for key 'bucket42': /content/bucket42.mp4
[WARN] Missing video file for key 'corridor11': /content/corridor11.mp4
[WARN] Missing video file for key 'corridor21': /content/corridor21.mp4
[WARN] Missing video file for key 'corridor22': /content/corridor22.mp4
[WARN] Missing video file for key 'door11': /content/door11.mp4
[WARN] Missing video file for key 'door22': /content/door22.mp4
[WARN] Missing video file for key 'door33': /content/door33.mp4
[WARN] Missing video file for key 'door41': /content/door41.mp4
[WARN] Missing video file for key 'door52': /content

printer31 frames: 100%|██████████| 119/119 [00:17<00:00,  6.97it/s]


[WARN] Missing video file for key 'printer32': /content/printer32.mp4
[WARN] Missing video file for key 'roomfire11': /content/roomfire11.mp4
[WARN] Missing video file for key 'roomfire22': /content/roomfire22.mp4
[WARN] Missing video file for key 'roomfire31': /content/roomfire31.mp4

[INFO] Skipping roomfire41 (excluded from training)
[WARN] Missing video file for key 'roomsmoke11': /content/roomsmoke11.mp4
[WARN] Missing video file for key 'roomsmoke22': /content/roomsmoke22.mp4
[WARN] Missing video file for key 'roomsmoke32': /content/roomsmoke32.mp4
[WARN] Missing video file for key 'roomsmoke42': /content/roomsmoke42.mp4
[WARN] Missing video file for key 'roomsmoke53': /content/roomsmoke53.mp4
[WARN] Missing video file for key 'smoking11': /content/smoking11.mp4
[WARN] Missing video file for key 'smoking12': /content/smoking12.mp4
[WARN] Missing video file for key 'smoking21': /content/smoking21.mp4
[WARN] Missing video file for key 'smoking22': /content/smoking22.mp4
[WARN] Miss

Saving printer31.mp4 to printer31 (1).mp4
Running detection on: /content/printer31 (1).mp4


/tmp/ipython-input-593449927.py:224: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  c = int(box.cls.cpu().numpy())
/tmp/ipython-input-593449927.py:225: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  conf = float(box.conf.cpu().numpy())



✅ Fast detection video saved at: /content/output_detection_fast.mp4
Download or play it from the Files panel.
